In [1]:
# from langchain.llms import OpenAI
from langchain_openai import OpenAI

In [5]:
import os
key = os.environ["OPENAI_API_KEY"]

In [3]:
llm = OpenAI(temperature=0.6, openai_api_key = key)

In [4]:
text = "what is the capital of India"

print(llm.invoke(text))



The capital of India is New Delhi.


In [8]:
from langchain import HuggingFaceHub
llm_huggingface = HuggingFaceHub(repo_id='google/flan-t5-large', model_kwargs={"temperature":0, "max_length":64}, huggingfacehub_api_token = hf)

c:\USERS\ANOSH\ONEDRIVE\DESKTOP\PROJECTS\CHABOT\CHATBOT\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [9]:
output = llm_huggingface.predict("whats the capital of Russia")
print(output)

c:\USERS\ANOSH\ONEDRIVE\DESKTOP\PROJECTS\CHABOT\CHATBOT\lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The method `BaseLLM.predict` was deprecated in langchain-core 0.1.7 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


Moscow


In [10]:
output = llm_huggingface.invoke("Act like a mobilechat assistant.. answer my query.. is my wifi on or off?")
print(output)

on


In [11]:
output = llm_huggingface.invoke("Can u turn on my wifi?")
print(output)

no


In [13]:
output = llm.invoke("Can u turn on my wifi?")
print(output)



Unfortunately, I am not able to turn on wifi as I am a text-based program and do not have access to physical devices. You will need to manually turn on your wifi through your device's settings or by using the physical button on your device.


PROMPT TEMPLATES

In [14]:
from langchain.prompts import PromptTemplate

prompt_template = PromptTemplate(input_variables=['country'],
                                 template= "Tell me the capital of this {country}")

prompt_template.format(country="India")

'Tell me the capital of this India'

In [15]:
from langchain_core.prompts import PromptTemplate

prompt_template = "Tell me the capital of {country}"
prompt = PromptTemplate(
    input_variables=["country"], template=prompt_template
)

In [16]:
chain = prompt | llm

In [17]:
print(chain.invoke("India"))

Combining Multiple chains using sequential chains

In [40]:
captial_prompt = "Tell me the capital of {country}"
prompt1 = PromptTemplate(
    input_variables=["country"], template=captial_prompt
)

captial_chain = prompt1 | llm

famous_prompt = "Suggest me some amazing places to visit in {capital}"
prompt2 = PromptTemplate(
    input_variables=['capital'], template=famous_prompt
)

famous_chain = llm | prompt2

famous_chain.invoke("New Delhi")



In [39]:
from langchain.chains import SimpleSequentialChain

chainoutput = SimpleSequentialChain(chains=[captial_chain, famous_chain])

chainoutput.invoke("India")

KeyError: 'chains'

CHATMODELS

In [1]:
from langchain.chat_models import ChatOpenAI

In [3]:
from langchain.schema import HumanMessage, SystemMessage, AIMessage

In [8]:
chatllm = ChatOpenAI(temperature=0.6, openai_api_key = key, model='gpt-3.5-turbo')

In [9]:
chatllm([
    SystemMessage(content="You are a phone ai assistant"),
    HumanMessage(content="Please provide me the steps to turn on my wifi")
])

c:\USERS\ANOSH\ONEDRIVE\DESKTOP\PROJECTS\CHABOT\CHATBOT\lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The method `BaseChatModel.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


AIMessage(content='Sure! To turn on your WiFi, follow these steps:\n\n1. Unlock your phone and go to the home screen.\n2. Open the Settings app. It usually looks like a gear icon.\n3. Look for the "Connections" or "Network & Internet" option and tap on it.\n4. Find the "WiFi" option and toggle the switch to turn it on. It may also be labeled as "Wireless & Networks" or something similar.\n5. Your phone will start scanning for available WiFi networks. Select your desired network and enter the password if required.\n6. Once connected, you should see the WiFi symbol appear in the status bar of your phone.\n\nThat\'s it! Your WiFi should now be turned on and connected to the network. Let me know if you need any further assistance.', response_metadata={'token_usage': {'completion_tokens': 162, 'prompt_tokens': 27, 'total_tokens': 189}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-79c2ff57-72f0-4837-a423-615481bdddd5-0')

OUTPUT PARSER

In [10]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts.chat import ChatPromptTemplate
from langchain.schema import BaseOutputParser

In [11]:
class commaseperatedoutput(BaseOutputParser):
    def parse(self, text:str):
        return text.strip().split(",")

In [12]:
template = "You are a helpful phone ai assistant. WHenever the user gives any input, you have to generate 5 synonymous words, in a comma seperated list"

In [15]:
human_template = "{text}"

chatprompt = ChatPromptTemplate.from_messages([
    ("system", template),
    ("human", human_template)]
)

In [16]:
chain = chatprompt|chatllm|commaseperatedoutput()

In [17]:
chain.invoke({"text":"intelligent"})

['smart', ' clever', ' knowledgeable', ' sharp', ' astute']